In [ ]:
import sys
sys.path.append('./ERNIE')
import numpy as np
from sklearn.metrics import f1_score
import paddle as P

from ernie.tokenizing_ernie import ErnieTokenizer
from ernie.modeling_ernie import ErnieModelForSequenceClassification

In [ ]:
#设置好所有的超参数，对于ERNIE任务学习率推荐取1e-5/2e-5/5e-5, 根据显存大小调节BATCH大小, 最大句子长度不超过512.
BATCH=32
MAX_SEQLEN=300
LR=5e-5
EPOCH=10


ernie = ErnieModelForSequenceClassification.from_pretrained('ernie-1.0', num_labels=3)
optimizer = P.optimizer.Adam(LR,parameters=ernie.parameters())
tokenizer = ErnieTokenizer.from_pretrained('ernie-1.0')

[INFO] 2021-11-12 17:41:25,477 [modeling_ernie.py:  267]:    get pretrain dir from https://ernie-github.cdn.bcebos.com/model-ernie1.0.1.tar.gz
INFO:filelock:Lock 140073509841552 acquired on /home/aistudio/.paddle-ernie-cache/466eabcffd6d6a83ae9cb97dd1a167bd.lock
INFO:filelock:Lock 140073509841552 released on /home/aistudio/.paddle-ernie-cache/466eabcffd6d6a83ae9cb97dd1a167bd.lock
W1112 17:41:25.554248   173 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W1112 17:41:25.559780   173 device_context.cc:465] device: 0, cuDNN Version: 7.6.
[INFO] 2021-11-12 17:41:34,646 [modeling_ernie.py:  285]:    loading pretrained model from /home/aistudio/.paddle-ernie-cache/466eabcffd6d6a83ae9cb97dd1a167bd
./ERNIE/ernie/modeling_ernie.py:296: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  log.warn('param:%s not set in pretrained model, skip' % k)
[WARNING] 2021-11-12 17:41:38,150 [modeling_ernie.py:  296]:    param:classifier.weight not set in pretrained model, skip
[WARNING] 2021-11-12 17:41:38,151 [modeling_ernie.py:  296]:    param:classifier.bias not set in pretrained model, skip
[INFO] 2021-11-12 17:41:38,601 [tokenizing_ernie.py:  102]:    get pretrain dir from https://ernie-github.cdn.bcebos.com/model-ernie1.0.1.tar.gz
INFO:filelock:Lock 140073507419280 acquired on /home/aistudio/.paddle-ernie-cache/466eabcffd6d6a83ae9cb97dd1a167bd.lock
INFO:filelock:Lock 140073507419280 released on /home/aistudio/.paddle-ernie-cache/466eabcffd6d6a83ae9cb97dd1a167bd.lock

In [ ]:
def make_data(path):
    data = []
    for i, l in enumerate(open(path)):
        if i == 0:
            continue
        l = l.strip().split('\t')
        text, label = l[0], int(l[1])
        text_id, _ = tokenizer.encode(text) # ErnieTokenizer 会自动添加ERNIE所需要的特殊token，如[CLS], [SEP]
        text_id = text_id[:MAX_SEQLEN]
        text_id = np.pad(text_id, [0, MAX_SEQLEN-len(text_id)], mode='constant') # 对所有句子都补长至300，这样会比较费显存；
        label_id = np.array(label+1)
        data.append((text_id, label_id))
    return data

train_data = make_data('./chnsenticorp/train/part.0')
test_data = make_data('./chnsenticorp/dev/part.0')

In [ ]:
def get_batch_data(data, i):
    d = data[i*BATCH: (i + 1) * BATCH]
    feature, label = zip(*d)
    feature = np.stack(feature)  # 将BATCH行样本整合在一个numpy.array中
    label = np.stack(list(label))
    feature = P.to_tensor(feature) # 使用to_variable将numpy.array转换为paddle tensor
    label = P.to_tensor(label)
    return feature, label

In [ ]:
for i in range(EPOCH):
    np.random.shuffle(train_data) # 每个epoch都shuffle数据以获得最佳训练效果；
    #train
    for j in range(len(train_data) // BATCH):
        feature, label = get_batch_data(train_data, j)
        loss, _ = ernie(feature, labels=label) # ernie模型的返回值包含(loss, logits)；其中logits目前暂时不需要使用
        loss.backward()
        optimizer.minimize(loss)
        ernie.clear_gradients()
        if j % 10 == 0:
            print('train %d: loss %.5f' % (j, loss.numpy()))
        # evaluate
        if j % 100 == 0:
            all_pred, all_label = [], []
            with P.no_grad(): # 在这个with域内ernie不会进行梯度计算；
                ernie.eval() # 控制模型进入eval模式，这将会关闭所有的dropout；
                for j in range(len(test_data) // BATCH):
                    feature, label = get_batch_data(test_data, j)
                    loss, logits = ernie(feature, labels=label) 
                    all_pred.extend(logits.argmax(-1).numpy())
                    all_label.extend(label.numpy())
                ernie.train()
            f1 = f1_score(all_label, all_pred, average='macro')
            acc = (np.array(all_label) == np.array(all_pred)).astype(np.float32).mean()
            print('acc %.5f' % acc)

train 0: loss 0.86285
acc 0.50084
train 10: loss 0.66456
train 20: loss 0.46766
train 30: loss 0.31338
train 40: loss 0.52718
train 50: loss 0.28027
train 60: loss 0.18098
train 70: loss 0.38587
train 80: loss 0.25202
train 90: loss 0.10612
train 100: loss 0.36271
acc 0.91216
train 110: loss 0.19755
train 120: loss 0.15493
train 130: loss 0.18104
train 140: loss 0.27483
train 150: loss 0.06898
train 160: loss 0.18187
train 170: loss 0.24634
train 180: loss 0.11194
train 190: loss 0.31004
train 200: loss 0.18383
acc 0.92990
train 210: loss 0.39319
train 220: loss 0.17911
train 230: loss 0.15745
train 240: loss 0.40289
train 250: loss 0.18617
train 260: loss 0.36464
train 270: loss 0.24336
train 280: loss 0.05639
train 290: loss 0.29951
train 0: loss 0.10927
acc 0.93074
train 10: loss 0.15014
train 20: loss 0.07078
train 30: loss 0.02730
train 40: loss 0.04488
train 50: loss 0.03658
train 60: loss 0.14192
train 70: loss 0.36672
train 80: loss 0.15562
train 90: loss 0.14741
train 100: loss 0.07760
acc 0.90541
train 110: loss 0.16851
train 120: loss 0.08833
train 130: loss 0.10430
train 140: loss 0.04092
train 150: loss 0.11011
train 160: loss 0.12601
train 170: loss 0.01970
train 180: loss 0.02083
train 190: loss 0.02704
train 200: loss 0.04330
acc 0.94172
train 210: loss 0.05111
train 220: loss 0.04243
train 230: loss 0.01082
train 240: loss 0.09999
train 250: loss 0.10365
train 260: loss 0.05661
train 270: loss 0.12513
train 280: loss 0.24553
train 290: loss 0.01133
train 0: loss 0.19247
acc 0.94679
train 10: loss 0.05103
train 20: loss 0.07351
train 30: loss 0.15348
train 40: loss 0.00540
train 50: loss 0.02154
train 60: loss 0.19600
train 70: loss 0.01916
train 80: loss 0.06111
train 90: loss 0.07352
train 100: loss 0.06478
acc 0.94426
train 110: loss 0.03744
train 120: loss 0.02267
train 130: loss 0.00507
train 140: loss 0.17992
train 150: loss 0.10633
train 160: loss 0.04254
train 170: loss 0.04541
train 180: loss 0.03623
train 190: loss 0.02851
train 200: loss 0.07782
acc 0.93666
train 210: loss 0.04136
train 220: loss 0.17985
train 230: loss 0.04763
train 240: loss 0.11269
train 250: loss 0.04453
train 260: loss 0.15736
train 270: loss 0.35397
train 280: loss 0.07969
train 290: loss 0.04847
train 0: loss 0.33155
acc 0.85642
train 10: loss 0.17980
train 20: loss 0.24665
train 30: loss 0.16764
train 40: loss 0.03118
train 50: loss 0.05369
train 60: loss 0.06388
train 70: loss 0.04429
train 80: loss 0.02393
train 90: loss 0.04675
train 100: loss 0.09336
acc 0.93412
train 110: loss 0.02698
train 120: loss 0.07860
train 130: loss 0.03140
train 140: loss 0.01014
train 150: loss 0.04252
train 160: loss 0.05134
train 170: loss 0.20238
train 180: loss 0.00942
train 190: loss 0.12322
train 200: loss 0.31296
acc 0.93497
train 210: loss 0.28249
train 220: loss 0.04357
train 230: loss 0.02566
train 240: loss 0.01707
train 250: loss 0.23882
train 260: loss 0.00347
train 270: loss 0.04164
train 280: loss 0.22989
train 290: loss 0.09394
train 0: loss 0.01540
acc 0.94764
train 10: loss 0.15776
train 20: loss 0.01940
train 30: loss 0.00642
train 40: loss 0.15567
train 50: loss 0.00355
train 60: loss 0.00412
train 70: loss 0.01525
train 80: loss 0.17096
train 90: loss 0.00169
train 100: loss 0.04087
acc 0.94510
train 110: loss 0.02118
train 120: loss 0.02053
train 130: loss 0.39884
train 140: loss 0.05539
train 150: loss 0.01432
train 160: loss 0.00613
train 170: loss 0.05895
train 180: loss 0.03971
train 190: loss 0.00688
train 200: loss 0.02204
acc 0.93834
train 210: loss 0.00522
train 220: loss 0.00888
train 230: loss 0.02209
train 240: loss 0.01645